# Assignment 7: Automated Machine Learning (Part 2)
## Objective:

As we learned from the class, the high demand for machine learning has produced a large amount of data scientists who have developed expertise in tools and algorithms. The features in the data will directly influence the results. However, it is tedious and unscalable to manually design and select features without domain knowledge. Thus, using some AutoML techniques will significantly help data scientists save labour and time. 
After completing this assignment, you should be able to answer the following questions:

1. Why do we need AutoML?
2. How does auto feature generation work?
3. How to use featuretools library to automatically generate features?
4. How to get useful features in a large feature space?

Imagine you are a data scientist in an online retailer company, for example, Amazon. Your task is to provide some recommendations to customers based on their historical purchase records.

In this assignment, we predict whether the customer will buy **Banana** in the next 4 weeks. It is a classification problem. To simplify the problem, we have already generated some features and provide the accuracy of the model (Random Forest model). The task for you is to generate **10** useful features and beat our model performance (AUC = 0.61, see below). 

For example, <br>
`MODE(orders.MODE(order_products.product_name)) = Bag of Organic Bananas` means whether the most frequent purchase of the customer is Bag of Organic Bananas. 

```
1: Feature: MODE(orders.MODE(order_products.product_name)) = Bag of Organic Bananas
2: Feature: MODE(order_products.aisle_id) is unknown
3: Feature: SUM(orders.NUM_UNIQUE(order_products.product_name))
4: Feature: MODE(orders.MODE(order_products.product_name)) = Boneless Skinless Chicken Breasts
5: Feature: MODE(order_products.product_name) = Boneless Skinless Chicken Breasts
6: Feature: STD(orders.NUM_UNIQUE(order_products.aisle_id))
7: Feature: MODE(order_products.aisle_id) = 83
8: Feature: MEDIAN(orders.MINUTE(order_time))
9: Feature: MODE(orders.DAY(order_time)) = 23
10: Feature: MODE(orders.MODE(order_products.department)) = produce

AUC 0.61
```


## Preliminary
If you never use featuretools before, you need to learn some basic knowledge of this topic. 
I found that these are some good resources: 
* [featuretools documentation](https://docs.featuretools.com/en/stable/)
* [Tutorial: Automated Feature Engineering in Python](https://towardsdatascience.com/automated-feature-engineering-in-python-99baf11cc219)

The data can be downloaded from [A7-2-data.zip](A7-2-data.zip). 

## 0. Preparation
Import relevant libraries and load the dataset: <br>
users: <br>
* user_id: customer identifier
* label:  1 if the customer will buy banana in next 4 weeks, 0 otherwise

orders: <br>
* order_id: order identifier
* user_id: customer identifier
* order_time: date of the order was placed on 

order_products: <br>
* order_id: order identifier
* order_product_id: foreign key
* reordered:  1 if this product has been ordered by this user in the past, 0 otherwise
* product_name: name of the product
* aisle_id: aisle identifier
* department: the name of the department
* order_time: date of the order was placed on

In [2]:
import pandas as pd
import numpy as np

import featuretools as ft
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import MinMaxScaler
import os
ft.__version__
pd.set_option('display.max_rows', None)

In [3]:
orders = pd.read_csv("orders.csv",index_col=[0])
order_products = pd.read_csv("order_products.csv",index_col=[0])
users = pd.read_csv("users.csv",index_col=[0])

print(users["label"].value_counts())
orders.shape

False    628
True     139
Name: label, dtype: int64


(5997, 3)

## Task 1. Feature Generation
In this task, you need to use featuretools to generate candidate features by using the above three tables.

### 1.1 Representing Data with EntitySet

Define entities and their relationships (see [https://docs.featuretools.com/en/stable/generated/featuretools.EntitySet.html](https://docs.featuretools.com/en/stable/generated/featuretools.EntitySet.html))

In [11]:
# Get the relationship between entities
def load_entityset(orders, order_products, users):
    # --- Write your code below ---
    # return the EntitySet object
    es = ft.EntitySet(id="product_data")
    es = es.entity_from_dataframe(entity_id="users",
                                  dataframe=users,
                                  index="user_id")
    es = es.entity_from_dataframe(entity_id="orders",
                                  dataframe=orders,
                                  index="order_id",
                                  time_index = 'order_time')
    es = es.entity_from_dataframe(entity_id="order_products",
                                  dataframe=order_products,
                                  index="order_product_id",
                                  time_index = 'order_time'
                                  )  
    r_users = ft.Relationship(es["users"]["user_id"],es["orders"]["user_id"])
    es = es.add_relationship(r_users)
    r_orders = ft.Relationship(es["orders"]["order_id"],es["order_products"]["order_id"])
    es = es.add_relationship(r_orders)
    return es

### 1.2 Deep Feature Synthesis

In [14]:
# Automatically generate features
es = load_entityset(orders, order_products, users)
# use ft.dfs to perform feature engineering
# --- Write your code below ---
feature_matrix, feature_defs = ft.dfs(entityset=es,
                                      target_entity="users")

In [15]:
feature_matrix_enc, features_enc = ft.encode_features(feature_matrix, feature_defs)
# output what features you generate
feature_matrix_enc

,label,COUNT(orders),SUM(order_products.aisle_id),SUM(order_products.order_new_id),SUM(order_products.reordered),SUM(order_products.order_prod_id),STD(order_products.aisle_id),STD(order_products.order_new_id),STD(order_products.reordered),STD(order_products.order_prod_id),...,MODE(order_products.orders.user_id) = 999,MODE(order_products.orders.user_id) = 998,MODE(order_products.orders.user_id) = 997,MODE(order_products.orders.user_id) = 996,MODE(order_products.orders.user_id) = 995,MODE(order_products.orders.user_id) = 994,MODE(order_products.orders.user_id) = 993,MODE(order_products.orders.user_id) = 992,MODE(order_products.orders.user_id) = 991,MODE(order_products.orders.user_id) is unknown
user_id,,,,,,,,,,,,,,,,,,,,,
1,False,4,1271,886118,11,886118,38.070486,7006.782478,0.511766,7006.782478,...,0,0,0,0,0,0,0,0,0,1
2,True,7,5419,2199713,29,2199713,39.596448,19711.051907,0.476918,19711.051907,...,0,0,0,0,0,0,0,0,0,1
3,False,5,2831,1457894,14,1457894,40.594305,7264.306780,0.480091,7264.306780,...,0,0,0,0,0,0,0,0,0,1
7,False,4,4781,1823942,36,1823942,35.379342,6460.798732,0.503413,6460.798732,...,0,0,0,0,0,0,0,0,0,1
10,False,4,7677,2007920,20,2007920,37.683787,10292.977642,0.382021,10292.977642,...,0,0,0,0,0,0,0,0,0,1
11,False,4,3500,888429,20,888429,40.517631,4351.201139,0.493089,4351.201139,...,0,0,0,0,0,0,0,0,0,1
13,False,9,4801,892602,33,892602,28.942257,3263.329605,0.496416,3263.329605,...,0,0,0,0,0,0,0,0,0,1
14,False,4,3078,1887868,7,1887868,42.048781,3908.892952,0.384808,3908.892952,...,0,0,0,0,0,0,0,0,0,1
15,False,6,1680,1047531,10,1047531,39.619462,1972.988387,0.512989,1972.988387,...,0,0,0,0,0,0,0,0,0,1


## Task 2. Feature Selection
In this task, you are going to select 10 features that are useful and train the *Random Forest* model. The goal is to beat the accuracy performance as we have shown before. Note that you have to use the Random Forest and the hyperparameters we provide in Section 2.2. In other words, your job is to achieve a higher AUC than 0.61 through feature generation/selection rather than through hyperparameter tuning or model selectoin. 

### 2.1 Select top features

In [16]:
# --- Write your code below ---
# Select top-10 features and return X, y (X.shape = (767, 10)
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2,f_classif,mutual_info_classif
from sklearn.impute import SimpleImputer

# Classifying label and feature into X and y 
X, y = feature_matrix_enc.iloc[:, 1:], feature_matrix_enc.iloc[:, 0]
# Using simple imputer to replace missing values with mean
mean_add = SimpleImputer(missing_values=np.nan, strategy='mean')
X = mean_add.fit_transform(X)
# Perform normalization of data
X = MinMaxScaler().fit_transform(X)
# Use select K best to pick top 10 features
selector = SelectKBest(mutual_info_classif, k=10)
X = selector.fit_transform(X, y)
print(X.shape)

(767, 10)


### 2.2 Get accuracy and list features

In [17]:
clf = RandomForestClassifier(n_estimators=400, n_jobs=-1)
scores = cross_val_score(estimator=clf,X=X, y=y, cv=3,
                             scoring="roc_auc", verbose=True)

print("AUC %.2f" % (scores.mean()))

# Print top-10 features
print("Top 10 features are as below")
index = selector.get_support()
for bool, feature in zip(index, features_enc[1:]):
    if bool:
        print(feature)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


AUC 0.76
Top 10 features are as below
<Feature: SUM(order_products.aisle_id)>
<Feature: NUM_UNIQUE(order_products.department)>
<Feature: MODE(order_products.product_name) = Banana>
<Feature: STD(orders.SKEW(order_products.order_new_id))>
<Feature: STD(orders.MEAN(order_products.reordered))>
<Feature: MAX(orders.NUM_UNIQUE(order_products.product_name))>
<Feature: MEAN(orders.SUM(order_products.aisle_id))>
<Feature: MODE(orders.MODE(order_products.product_name)) = Bag of Organic Bananas>
<Feature: MODE(orders.MODE(order_products.product_name)) = Banana>
<Feature: MODE(orders.WEEKDAY(order_time)) = 2>


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    8.0s finished


## Task3. Writing Questions

1. Please list three advantages and disadvantages of featuretools. 
2. For those disadvantages you listed above, do you have any idea to improve it? 

--- Write your answer here---
1. Advantages and disadvantages of feature tools are as below:
Advantages:
a. Reduces development time that are spend on feature engineering
b. Can provide better predictive performance
c. Provides interpretable features with real world significance

Disadvantages
a. DFS in feature tools is not capable of creating aggregation features with fewer than 2 entities. Featuretools looks for a relationship, and aggregates based on that relationship.
b. Feature tools can be run on those datasets that can be accomodated to the memory of a single machine.
c. For supervised machine learning, we need to supply lables with our own labels. 

2. Ideas to improvide the disadvantages:
a. To generate aggregation features with a single entity, we could probably perform a normalization of the existing entity to create two or more entities. This would help us to build a relationship between newly created entities to further perform aggregation and generation of features.
b. Memory management can be done effectively by setting chunksize ,by partitioning the data based on key attributes or by reducing the number of unique cut off times.
c. In-order to address label issues, Featurelabs also has a tool called 'Compose' that can automatically generate labels with cut-off times.

## Submission
Complete the code in this notebook, and submit it to the CourSys activity Assignment 7.